#### Toto, je původní soubor z registru vozidel (velká velikost 18 GB), ze souboru jsme identifikovali chybně vyplněné řádky (do sloupců Plně elektrické to natáhlo zdvihový objem), které jsme smazali a vygenerovali nový výstupní soubor csv. "RSV_bez_chybnych_radku.csv bez těchto chybných řádků + byly definovány pouze potřebné sloupce

In [ ]:
import pandas as pd
import csv
import numpy as np

"""Precte po chuncich CSV, vyfiltruje sloupce, vyfiltruje radky podle hodnot ve sloupci 'Plně elektrické vozidlo' 
(pokud tam jsou jine nez "ANO", "NE" a NaN, tak je zahodi (overit, ze ponechava NaN! Pokud ho vlastne chceme)."""

chunk_size = 100_000
output_file_path = "06_RSV_bez_chybnych_radku.csv"
bad_values_count = 0
is_first_chunk = True

for chunked_csv_reader in pd.read_csv(
# chunked_csv_reader = pd.read_csv(
    "RSV_vypis_vozidel_20250301.csv",
    usecols=[
        "Datum 1  registrace", "Datum 1  registrace v ČR", "Druh vozidla", "Kategorie vozidla",
        "Tovární značka", "Palivo", "Plně elektrické vozidlo",
        "Hybridní vozidlo", "PČV", "Rok výroby"
    ],
    sep=",",
    on_bad_lines="warn",  # budeme alespon znat, kolik radku to preskoci pri nacitani
    quoting=csv.QUOTE_MINIMAL,  # vyzkouset jine quotovani? Viz https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html a https://docs.python.org/3/library/csv.html#csv.QUOTE_ALL
    encoding="utf-8",
    low_memory=False,
    chunksize=chunk_size):  # <- tady říkáme "čti po 100 000 řádcích"

    
    
    # for chunk in chunked_csv_reader:
        good_values_mask = chunked_csv_reader['Plně elektrické vozidlo'].isin(['NE', 'ANO', np.nan])  # overit, ze np.nan je spravna hodnota
        bad_values_mask = ~good_values_mask
        bad_values_count += bad_values_mask.sum()
        filtered_chunk = chunked_csv_reader[good_values_mask]
        if filtered_chunk.empty:
            continue  # nikam nezapisujeme
        
        filtered_chunk.to_csv(
            output_file_path,
            mode='w' if is_first_chunk else 'a',  # prvni chunk zapiseme ("write"), dalsi uz jen "appendujeme"
            header=is_first_chunk,
            index=False
        )
        is_first_chunk = False
    
print(bad_values_count)

68676


In [11]:
!code RSV_sloupce.txt  #vyjede mi ve vedlejší záložce všecky sloupce